In [68]:
#Library imports

import numpy as np

In [69]:
#Hidden Layer
#Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))


    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)

In [70]:
#Activation Functions
#Included here are the functions for both the forward and backward pass

#Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

#Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

#TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

#ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [71]:
#Loss Functions

class Loss:
    #Calculate the data and regualarization losses
    #Given the model output and group truth/target values
    def calculate(self, output, y):
        #Calculate sample losses
        sample_losses = self.forward(output, y)
        #Calculate the mean loss
        data_loss = np.mean(sample_losses)
        #Return the mean loss
        return data_loss

#MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        #Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        #Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        #Normalize the gradient over samples
        self.dinputs = self.dinputs / samples

#Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        #Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1e-7)
        #Calculate Binary Cross Entropy
        return - (y_true *np.log(y_pred_clipped) + (1 -y_true) * np.log(1 - y_pred_clipped))
    
    def backward(self, y_pred, y_true):
        #Gradient of BCE Loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        #Normalize the gradient over samples
        self.dinputs = self.dinputs / samples

    
#Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    #Forward pass
    def forward(self, y_pred, y_true):
        #Number of samples in a batch
        samples = y_pred.shape[0]

        #Clip data to prevent division by 0
        #Clip both sides to not drage mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        #Probabilities for target values
        #Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        #Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences =  np.sum(y_pred_clipped * y_true, axis = 1)

        #Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    #Backward pass
    def backward(self, dvalues, y_true):
        #Number of samples
        samples = len(dvalues)
        #Number of labels in every sample
        #Use the first sample to count them
        labels = len(dvalues[0])

        #Check if labels are sparse, turn them into one-hot vector values
        #the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        #Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

In [72]:
#Start of Optimizers

class Optimizer_SGD:
    # Initialize the optimizer - default learning rate to 1
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0, adaptive=None, rho=0.9, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        self.adaptive = adaptive
        self.rho = rho
        self.epsilon = epsilon

    def pre_update_params(self):
        # compute decayed learning rate for this iteration
        if self.decay:
            self.current_learning_rate = self.learning_rate / (1.0 + self.decay * self.iterations)
        else:
            self.current_learning_rate = self.learning_rate

    # Update the parameters
    def update_params(self, layer):
        # Initialize momentum & cache arrays if they don't exist
        if not hasattr(layer, 'weight_momentum'):
            layer.weight_momentum = np.zeros_like(layer.weights)
            layer.bias_momentum = np.zeros_like(layer.biases)
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Adaptive gradient methods
        if self.adaptive == 'adagrad':
            layer.weight_cache += layer.dweights ** 2
            layer.bias_cache += layer.dbiases ** 2
            grad_w = layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
            grad_b = layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)
        elif self.adaptive == 'rmsprop':
            layer.weight_cache = self.rho * layer.weight_cache + (1 - self.rho) * (layer.dweights ** 2)
            layer.bias_cache = self.rho * layer.bias_cache + (1 - self.rho) * (layer.dbiases ** 2)
            grad_w = layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
            grad_b = layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)
        else:
            grad_w = layer.dweights
            grad_b = layer.dbiases

        # momentum (velocity) update
        if self.momentum:
            layer.weight_momentum = self.momentum * layer.weight_momentum - self.current_learning_rate * grad_w
            layer.bias_momentum = self.momentum * layer.bias_momentum - self.current_learning_rate * grad_b
            layer.weights += layer.weight_momentum
            layer.biases += layer.bias_momentum
        else:
            layer.weights += -self.current_learning_rate * grad_w
            layer.biases += -self.current_learning_rate * grad_b

    def post_update_params(self):
        self.iterations += 1


In [73]:
#Spiral Data
import nnfs
from nnfs.datasets import spiral_data

#Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

#print (X[:5])
#print (X.shape)
#print (y[:5])
#print (y.shape)

In [74]:
#Iris Dataset
#From the scikit-learn library
#from sklearn.datasets import load_iris
#iris = load_iris()
#X = iris.data # features
#y = iris.target # target labels

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [75]:
# Neural Network initialization
# Create a Dense Layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Make sure you check the shape of the features, in order to adjust the input size of the first layer
# dense1 = Layer_Dense(4, 3)

# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()

# Create a 2nd dense layer with 3 input and 3 output values
dense2 = Layer_Dense(3, 3)

# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()

# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

# Create the optimizer
optimizer = Optimizer_SGD()

In [76]:
# Perform a forward pass of our training data
# give the input from the dataset to the first layer
dense1.forward(X)

# Activation function
activation1.forward(dense1.output)

# Pass on the 2nd layer
dense2.forward(activation1.output)

activation2.forward(dense2.output)

# Calculate the loss
loss_function.forward(activation2.output, y)

# Check the model's performance
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

# Print the accuracy
print('acc:', accuracy)

acc: 0.3933333333333333


In [77]:
# Perform a backward pass of our training data
# From loss to 2nd softmax activation
loss_function.backward(activation2.output, y)
dvalues = loss_function.dinputs # Gradient of the loss w.r.t softmax output

print(dvalues.shape)
# print(dvalues)

# From 2nd softmax to 2nd dense layer
activation2.backward(dvalues)
# From 2nd dense layer to 1st ReLU activation
dense2.backward(activation2.dinputs)

# From 1st ReLU activation to 1st dense layer
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

(300, 3)


In [78]:
# Check the gradient values of the weights and biases of the established layers
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)


# Update the weights and biases
optimizer.update_params(dense1)
optimizer.update_params(dense2)

[[ 1.33856354e-04  1.56926693e-04  3.17680352e-05]
 [ 1.93095371e-04 -2.83564210e-04  7.68290972e-05]]
[[-0.00018032 -0.00053259 -0.00010286]]
[[-1.67655819e-04  9.53486121e-05  7.23072067e-05]
 [ 2.06791592e-04 -1.17029758e-04 -8.97618337e-05]
 [-2.17264990e-04  1.71160482e-04  4.61045085e-05]]
[[-1.75240213e-05  6.05167449e-06  1.14723468e-05]]


In [79]:
for epoch in range(1, 1001):                    # epochs 1..1000
    optimizer.pre_update_params()               # update current_learning_rate before FP/BP

    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    # Loss & accuracy
    sample_losses = loss_function.forward(activation2.output, y)
    data_loss = np.mean(sample_losses)
    predictions = np.argmax(activation2.output, axis=1)
    y_true = y if y.ndim == 1 else np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y_true)

    if epoch % 100 == 0 or epoch == 1000:
        print(f'epoch: {epoch}, acc: {accuracy:.3f}, loss: {data_loss:.3f}, lr: {optimizer.current_learning_rate:.5f}')

    # Backward pass
    loss_function.backward(activation2.output, y)   # sets loss_function.dinputs
    dvalues = loss_function.dinputs
    activation2.backward(dvalues)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)            # use dense2.dinputs 
    dense1.backward(activation1.dinputs)

    # Update parameters
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()                  # increment optimizer.iterations

epoch: 100, acc: 0.417, loss: 1.097, lr: 1.00000
epoch: 200, acc: 0.397, loss: 1.078, lr: 1.00000
epoch: 300, acc: 0.393, loss: 1.075, lr: 1.00000
epoch: 400, acc: 0.413, loss: 1.072, lr: 1.00000
epoch: 500, acc: 0.423, loss: 1.071, lr: 1.00000
epoch: 600, acc: 0.433, loss: 1.071, lr: 1.00000
epoch: 700, acc: 0.433, loss: 1.071, lr: 1.00000
epoch: 800, acc: 0.433, loss: 1.071, lr: 1.00000
epoch: 900, acc: 0.433, loss: 1.071, lr: 1.00000
epoch: 1000, acc: 0.437, loss: 1.071, lr: 1.00000
